<h2> First Approach: TF/IDF </h2>

In [1]:
# --- Preprocessing ---
import pandas as pd
from sklearn.utils import resample

# --- Processing ---
import re
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder

# --- Postprocessing ---
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
# Reading the files
train = pd.read_csv('Datasets/1train_data.csv',encoding='ISO-8859-1')
test = pd.read_csv('Datasets/1test_data.csv',encoding='ISO-8859-1')
pd.set_option('display.max_colwidth', -1)

print('Column names: %s'%(train.columns))
print('Train length: %s'%(len(train)))
print('Test length: %s '%(len(test)))

Column names: Index(['sentiment', 'content'], dtype='object')
Train length: 30000
Test length: 10000 


In [3]:
train.tail()

,sentiment,content
29995,happiness,I had a great date last night...tried to find the CDCaves with Daniel it was HILARIOUSLY FUN!!!
29996,sadness,With alex
29997,happiness,@fureousangel that is comedy good luck my friend!
29998,fun,"stephs grad party gr8! shoved cake in her face, watchd sis bitch slap a boy, ate good food satisfied"
29999,happiness,@jesfive SWEEEEET - San Fran is awesome!!!! Love it there


In [4]:
# Tweets cleaning
def clean_text(df,text_field):
    df[text_field] = df[text_field].str.lower()
    mystring = r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"
    # the first filter is capture everything eclosed that starts with an @
    # and that has any single character from the range A-Z or a-z or 0-9 with more
    # than one element inside of that
    # the second filter is capture everything that is not a number, a lower or upper
    # case letter after the last line
    df[text_field] = df[text_field].apply(lambda element: re.sub(mystring,"",element))
    return df

train_clean = clean_text(train,'content')
test_clean = clean_text(test,'content')
train_clean.tail()

,sentiment,content
29995,happiness,i had a great date last nighttried to find the cdcaves with daniel it was hilariously fun
29996,sadness,with alex
29997,happiness,that is comedy good luck my friend
29998,fun,stephs grad party gr8 shoved cake in her face watchd sis bitch slap a boy ate good food satisfied
29999,happiness,sweeeeet san fran is awesome love it there


In [6]:
myfile = [k for k in train_clean['content'] if k.strip() != '']

In [8]:
from bert_serving.client import BertClient
bc = BertClient()
print('BERT connected')
data = bc.encode(myfile)

BERT connected


/anaconda3/lib/python3.7/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [9]:
import pickle

with open('bert_emb_1train_data.pk', mode='wb') as f:
    pickle.dump(data,f)

In [10]:
with open('bert_emb_1train_data.pk', mode='rb') as f:
    loaded_data = pickle.load(f)

In [12]:
loaded_data.shape

(29938, 768)

In [ ]:
print('Train length: %s'%(len(train)))
print('Test length: %s '%(len(test)))

In [ ]:
sentiments = list(set(train_clean['sentiment']))
how_many_sentiments = len(sentiments)
print('Sentiment labels: %s \n\nNumber of sentiments: %s '%(sentiments,how_many_sentiments))

In [ ]:
le = LabelEncoder()
train_clean['target'] = le.fit_transform(train_clean['sentiment'])
train_clean.tail()

In [ ]:
print(le.classes_)

<h3>Upsampling</h3>

In [ ]:
# Number of non-racist/non-sexist (0's) tweets
# in the training
lista = [train_clean[train_clean.target==index] for index in list(set(train_clean['target']))]
lista_dos = [len(k) for k in lista]
mymap = dict(zip(le.classes_,lista_dos))
print('Number of non-racist/non-sexist tweets: \n%s '%(mymap))

In [ ]:
# Number of racist/sexist (1's) tweets in the training
train_minority = train_clean[train_clean.label==1]
print('Number of racist/sexist tweets: %s '%(len(train_minority)))

In [ ]:
# Upsampling:
# We use the tags with less frequency, in this case 1's, inside
# the training in order to create an upsampled training set with
# the same dimension the tags with the highest frequency have (in
# this case 0's), allowing tweet repetition.
train_minority_upsampled = resample(train_minority, # data  set to use
                                    replace=True, # repetitions are allowed
                                    n_samples=len(train_majority), # I want 29720 samples
                                    random_state=123) # using this random state

# Now that the minority of tweets is no longer outmatched by 
# the majority, we can put them together, obtaining a training
# set of twice the size of the majority, i,.e. 2*(2970) = 59440
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
print('Upsampling')
print('Number of total tweets in training: %s,%s '%((train_upsampled.shape)))
print(train_upsampled['label'].value_counts())

<h3>Downsampling</h3>

In [ ]:
# Number of non-racist/non-sexist (0's) tweets
# in the training
train_majority = train_clean[train_clean.label==0]
print('Number of non-racist/non-sexist tweets: %s '%(len(train_majority)))

In [ ]:
# Number of racist/sexist (1's) tweets in the training
train_minority = train_clean[train_clean.label==1]
print('Number of non-racist/non-sexist tweets: %s '%(len(train_minority)))

In [ ]:
# Downsampling:
# We use the tags with the largest frequency, in this case the 
# 0's inside the training in order to create an downsampled training 
# set with the same dimension the tags with the lowest frequency have (in
# this case 1's), allowing tweet repetition.
train_majority_downsampled = resample(train_majority, # use the train majority
                                 replace=True,  # repetitions are allowed
                                 n_samples=len(train_minority), # I want 2242 samples
                                 random_state=123) #using this random state

# Now that the majority of tweets has been reduced to a set
# with the same size as the minority, we can put them together,
# obtaining a training set of twice the size of the minority, i,.e. 2*(2242) = 4484
train_downsampled = pd.concat([train_majority_downsampled, train_minority])
print('Downsampling')
print('Number of total tweets in training: %s,%s '%((train_downsampled.shape)))
print(train_downsampled['label'].value_counts())

<h3> Model Training </h3>

In [ ]:
# Training and test sets
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],
                                                    random_state=0)

# Format: Training
#
# 1. Vectorizer 
# Here we get the list of words of the whole 
# training set, the whole tokenized version 
# of the dataset.
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)

# 2. TF/IDF
# It indicates how many times a word appears
# inside each tweet.
tfidf_transformer = TfidfTransformer()#use_idf=False).fit(X)
X_train_tfidf = tfidf_transformer.fit_transform(X)

# The Model
clf = SGDClassifier()
model = SGDClassifier().fit(X_train_tfidf,y_train)

# Format: Testing
# 
# 1. Vectorizer 
X_prime = vectorizer.transform(X_test)
# 2. TF/IDF
X_test_tfidf = tfidf_transformer.transform(X_prime)

# Prediction
y_predict = model.predict(X_test_tfidf)
f1_score(y_test, y_predict)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_predict))#,target_names=twenty_test.target_names))

<h2> Pipeline </h2>

In [ ]:
# Pipelines:
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier()),
])

<h2> Prediction (Upsampling) </h2>

In [ ]:
# Test and Training Splitting
X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],
                                                    random_state=0)
# Model
model_up = pipeline_sgd.fit(X_train_up, y_train_up)
model_up

y_predict_up = model_up.predict(X_test_up)
y_predict_up
print('Accuracy with upsampling: %s '%(f1_score(y_test_up, y_predict_up)))

<h2> Prediction (Downsampling) </h2>

In [ ]:
# Test and Training Splitting
X_train_down, X_test_down, y_train_down, y_test_down = train_test_split(train_downsampled['tweet'],
                                                    train_downsampled['label'],
                                                    random_state=0)
# Model
model_down = pipeline_sgd.fit(X_train_down, y_train_down)
model_down

y_predict_down = model_up.predict(X_test_down)
y_predict_down
print('Accuracy with downsampling: %s '%(f1_score(y_test_down, y_predict_down)))

In [ ]:
# # scikit-learn bootstrap
# from sklearn.utils import resample
# # data sample
# data = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
# print('data: %s'%(data))
# # prepare bootstrap sample
# boot = resample(data, replace=True, n_samples=2*len(data), random_state=1)
# print('Bootstrap Sample: %s' % boot)
# # out of bag observations
# oob = [x for x in data if x not in boot]
# # print('OOB Sample: %s' % oob)

# # data: [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
# # Bootstrap Sample: [0.6, 0.4, 0.5, 0.1, 0.2, 0.4, 0.6, 0.1, 0.1, 0.2, 0.5, 0.6]
# # OOB Sample: [0.3]

In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
print(corpus)

In [ ]:
myvectorizer = CountVectorizer()
myX = myvectorizer.fit_transform(corpus)
print(myvectorizer.get_feature_names())

In [ ]:
print(myX)

In [ ]:
# ['This is the first document.', 'This document is the second document.', 'And this is the third one.', 'Is this the first document?']
# ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
# [[0 1 1 1 0 0 1 0 1]
#  [0 2 0 1 0 1 1 0 1]
#  [1 0 0 1 1 0 1 1 1]
#  [0 1 1 1 0 0 1 0 1]]

mytf_transformer = TfidfTransformer(use_idf=False).fit(myX)
print(mytf_transformer)

In [ ]:
myX_train_tf = mytf_transformer.transform(myX)
print(myX_train_tf)

In [ ]:
myX_train_tf.toarray()
# TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False,
#          use_idf=False)
#   (0, 1)	0.4472135954999579
#   (0, 2)	0.4472135954999579
#   (0, 3)	0.4472135954999579
#   (0, 6)	0.4472135954999579
#   (0, 8)	0.4472135954999579
#   (1, 1)	0.7071067811865475
#   (1, 3)	0.35355339059327373
#   (1, 5)	0.35355339059327373
#   (1, 6)	0.35355339059327373
#   (1, 8)	0.35355339059327373
#   (2, 0)	0.4082482904638631
#   (2, 3)	0.4082482904638631
#   (2, 4)	0.4082482904638631
#   (2, 6)	0.4082482904638631
#   (2, 7)	0.4082482904638631
#   (2, 8)	0.4082482904638631
#   (3, 1)	0.4472135954999579
#   (3, 2)	0.4472135954999579
#   (3, 3)	0.4472135954999579
#   (3, 6)	0.4472135954999579
#   (3, 8)	0.4472135954999579
# (4, 9)

# myclf = SGDClassifier()#loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)